In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import layers, models
import keras

In [2]:
# Define path to the dataset
PATH = "C:/Users/emanu/Universidade/Mestrado/1_Ano/2_Semestre/ACA/Projeto/ACA-Project/AML-Project2/dataset"

# Read the CSV file containing the image paths and labels
# Read the training set data
train_data = np.load(os.path.join(PATH, "trainX.npy"))
train_labels = np.load(os.path.join(PATH, "trainy.npy"), allow_pickle=True)

test_data = np.load(os.path.join(PATH, "testX.npy"))

print(train_data.shape)
display(train_data)

(12000, 50, 50, 3)


array([[[[ 47,  64,  77],
         [ 43,  57,  80],
         [182, 206, 224],
         ...,
         [ 10,   8,   8],
         [  9,   7,   7],
         [  7,   5,   5]],

        [[ 42,  59,  80],
         [ 14,  22,  35],
         [ 50,  62,  74],
         ...,
         [ 14,   9,  10],
         [ 10,   5,   6],
         [ 11,   6,   7]],

        [[ 51,  68,  87],
         [  0,   5,   9],
         [  7,  14,  23],
         ...,
         [ 13,   8,  10],
         [ 13,   8,   9],
         [ 11,   6,   7]],

        ...,

        [[ 57,  63,  76],
         [ 43,  55,  65],
         [ 33,  45,  63],
         ...,
         [ 37,  36,  40],
         [ 17,  12,  14],
         [ 16,  11,  13]],

        [[ 48,  58,  65],
         [ 47,  53,  66],
         [ 45,  53,  60],
         ...,
         [ 19,  21,  22],
         [ 25,  20,  22],
         [ 14,   9,  10]],

        [[ 32,  40,  47],
         [ 13,  19,  30],
         [ 11,  17,  24],
         ...,
         [237, 239, 247],
        

In [3]:
train_data = train_data / 255.0

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, train_size=0.8, random_state=42, stratify=train_labels)

In [5]:
display(X_train.shape)

(9600, 50, 50, 3)

In [6]:
model = models.Sequential()

model.add(layers.Conv2D(32, 3, activation='relu', input_shape=(50, 50, 3)))
model.add(layers.MaxPool2D())
model.add(layers.Conv2D(64, 3, activation='relu'))
model.add(layers.MaxPool2D())
model.add(layers.Conv2D(128, 3, activation='relu'))
model.add(layers.MaxPool2D())
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 11, 11, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 4, 128)        0

In [7]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)

In [8]:
train_Y_one_hot = keras.utils.to_categorical(y_train)

In [9]:
model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x=X_train,
    y=train_Y_one_hot,
    batch_size=32,
    epochs=20,
    validation_split=0.1
)

Epoch 1/20
270/270 [==============================] - 9s 30ms/step - loss: 1.1937 - accuracy: 0.5068 - val_loss: 0.9888 - val_accuracy: 0.6198
Epoch 2/20
270/270 [==============================] - 8s 30ms/step - loss: 0.9142 - accuracy: 0.6449 - val_loss: 0.8426 - val_accuracy: 0.6625
Epoch 3/20
270/270 [==============================] - 8s 30ms/step - loss: 0.7859 - accuracy: 0.7009 - val_loss: 0.7314 - val_accuracy: 0.7396
Epoch 4/20
270/270 [==============================] - 8s 30ms/step - loss: 0.6948 - accuracy: 0.7470 - val_loss: 0.6934 - val_accuracy: 0.7594
Epoch 5/20
270/270 [==============================] - 8s 30ms/step - loss: 0.6173 - accuracy: 0.7809 - val_loss: 0.6287 - val_accuracy: 0.7844
Epoch 6/20
270/270 [==============================] - 8s 30ms/step - loss: 0.5676 - accuracy: 0.7980 - val_loss: 0.7276 - val_accuracy: 0.7333
Epoch 7/20
270/270 [==============================] - 8s 30ms/step - loss: 0.5184 - accuracy: 0.8118 - val_loss: 0.5964 - val_accuracy: 0.8062

In [11]:
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
test_Y_one_hot = keras.utils.to_categorical(y_test)
test_Y_one_hot = tf.convert_to_tensor(test_Y_one_hot, dtype=tf.float32)


from sklearn.metrics import classification_report

model.evaluate(X_test, test_Y_one_hot)

pred = model.predict(X_test)
pred = np.argmax(pred, axis=1)

test_Y_one_hot = np.argmax(test_Y_one_hot, axis=1)

print(pred)
print(test_Y_one_hot)

print(classification_report(test_Y_one_hot, pred, zero_division=0))
print('=========================================')

75/75 [==============================] - 1s 9ms/step
[1 0 3 ... 1 1 2]
[1 0 3 ... 1 1 0]
              precision    recall  f1-score   support

           0       0.78      0.82      0.80       400
           1       0.95      0.95      0.95       400
           2       0.77      0.83      0.80       400
           3       0.79      0.81      0.80       400
           4       0.85      0.79      0.82       400
           5       0.86      0.78      0.82       400

    accuracy                           0.83      2400
   macro avg       0.83      0.83      0.83      2400
weighted avg       0.83      0.83      0.83      2400



In [179]:
test_df = pd.read_csv(os.path.join(PATH, "test_intel-image-classification-csvdata-kaggle.csv"))

test_data = test_data / 255.0

test_data = tf.convert_to_tensor(test_data, dtype=tf.float32)

In [180]:
preds = model.predict(test_data)
predict_test = np.argmax(preds, axis=1)

submission = pd.DataFrame({'number': test_df['number'], 'class': predict_test})
print(submission)
submission.to_csv('submission_cnn_oneHot_01val_40e_64b_maxp_025do_.csv', index=False)

158/158 [==============================] - 2s 12ms/step
      number  class
0          0      1
1          1      0
2          2      0
3          3      0
4          4      0
...      ...    ...
5046    5046      5
5047    5047      5
5048    5048      5
5049    5049      5
5050    5050      5

[5051 rows x 2 columns]
